In [1]:
from torch_geometric.data import Data

import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, Adagrad
from torch_geometric.utils import degree
import pickle
from torch import optim

from torch_geometric.loader import DataLoader

import numpy as np
import os
import time
import argparse
import scipy

# Create a configuration for position encodings (including SignNet)
from yacs.config import CfgNode as CN
from long_range_graph.position_enc.posenc_config import set_cfg_posenc
from long_range_graph.position_enc.signnet_pos_encoder import SignNetNodeEncoder

/home/zluo/anaconda3/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/zluo/anaconda3/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


In [2]:
# Dataset
from VLSI_ML.src.data.netlist import *
#from long_range_graph.position_enc.posenc_stats import compute_posenc_stats

In [3]:
from torch_geometric.transforms.add_positional_encoding import AddLaplacianEigenvectorPE

In [4]:
import sys # caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'long_range_graph/models/pyg/')
from gnn import GNN

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd

In [7]:
os.chdir("VLSI_ML/")

In [33]:
dataset = netlist_v2("/home/zluo/nn/gnn_base_2/")
#dataset.process()

In [37]:
config = CN()
config = set_cfg_posenc(config)
config.posenc_SignNet.model = 'MLP'
config.posenc_SignNet.post_layers = 1
config.posenc_SignNet.eigen.max_freqs = 20

In [38]:
train_idx = list(np.load("data/X_train_idx.npy"))
valid_test_idx = list(np.load("data/X_test_idx.npy"))
valid_idx = valid_test_idx
test_idx = valid_test_idx

In [39]:
train_data = dataset[train_idx]
valid_data = dataset[valid_idx]
test_data = dataset[test_idx]

In [40]:
batch_size = 1

In [41]:
train_dataloader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
valid_dataloader = DataLoader(valid_data, batch_size = batch_size, shuffle = False)
test_dataloader = DataLoader(test_data, batch_size = batch_size, shuffle = False)

In [42]:
device = torch.device('cuda')

In [50]:
model = GNN(gnn_type = 'gcn', num_tasks = 1, virtual_node = False, num_layer = 3, emb_dim = 64,
            use_signnet = True, node_dim = 5, cfg_posenc = config, device=device, drop_ratio=0.5).to(device)

2
ModuleList(
  (0): Linear(in_features=1, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=64, bias=True)
)
2
ModuleList(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=64, bias=True)
)
2
ModuleList(
  (0): Linear(in_features=64, out_features=64, bias=True)
  (1): Linear(in_features=64, out_features=4, bias=True)
)

4 20

1
ModuleList(
  (0): Linear(in_features=80, out_features=16, bias=True)
)


In [51]:
def square_relative_error(output, target):
    loss = (torch.square(output - target))/(torch.square(target))
    return loss

def mean_relative_error(output, target):
    loss = torch.absolute(output - target)/torch.absolute(target)
    return loss

def mean_abs_error(output, target):
    loss = torch.absolute(output - target)
    return loss

def mse(output, target):
    loss = torch.square(output - target)
    return loss

In [45]:
model = torch.load("train_signnet_util.pt")

In [52]:
optimizer = Adam(model.parameters(), lr = 0.001)

In [53]:
for g in optimizer.param_groups:
    g['lr'] = g['lr']*0.5

In [ ]:
num_epoch = 100

for epoch in range(num_epoch):
    model.train()
    total_loss = 0.0
    nBatch = 0

    for batch_idx, data in enumerate(train_dataloader):
        data = data.to(device)
        target = data.y.to(device)
        predict = model(data, data.stats)
        optimizer.zero_grad()
        loss = F.mse_loss(predict.view(-1), target, reduction = 'mean')
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        nBatch += 1
        if batch_idx % 100 == 0:
            print('Batch', batch_idx, '/', len(train_dataloader),': Loss =', total_loss/nBatch)
            
    print(total_loss/len(train_dataloader))
    optimizer.zero_grad()
    
    model.eval()
    output_lst = []
    target_lst = []

    for batch_idx, data in enumerate(valid_dataloader):
        data = data.to(device = device)
        target = data.y.to(device)
        predict = model(data, data.stats)
        output_lst.append(predict.view(-1).item())
        target_lst.append(target.item())

    orig_out = torch.FloatTensor(output_lst)
    orig_target = torch.FloatTensor(target_lst)

    eval_lst = [square_relative_error, mean_relative_error, mean_abs_error, mse]
    for evals in eval_lst:
        print(torch.mean(evals(orig_out, orig_target)))

Batch 0 / 1744 : Loss = 99220.078125
Batch 100 / 1744 : Loss = 20573787.93125387
Batch 200 / 1744 : Loss = 22473891.50334752
Batch 300 / 1744 : Loss = 22616822.669581417
Batch 400 / 1744 : Loss = 23732664.77871742
Batch 500 / 1744 : Loss = 23446424.938713636
Batch 600 / 1744 : Loss = 24038796.26976625
Batch 700 / 1744 : Loss = 23672833.6127961
Batch 800 / 1744 : Loss = 24007152.12922781
Batch 900 / 1744 : Loss = 24523759.968297824
Batch 1000 / 1744 : Loss = 24923679.618187293
Batch 1100 / 1744 : Loss = 25187648.013304997
Batch 1200 / 1744 : Loss = 24984371.78059574
Batch 1300 / 1744 : Loss = 25068791.388251245
Batch 1400 / 1744 : Loss = 25041972.951529894
Batch 1500 / 1744 : Loss = 25014160.642072726
Batch 1600 / 1744 : Loss = 24904990.37962186
Batch 1700 / 1744 : Loss = 24633324.338389635
24419101.797212757
tensor(0.0040)
tensor(0.0517)
tensor(4151.8037)
tensor(27519130.)
Batch 0 / 1744 : Loss = 7815300.5
Batch 100 / 1744 : Loss = 24594352.06856484
Batch 200 / 1744 : Loss = 24134923.8

Batch 600 / 1744 : Loss = 20562621.413177744
Batch 700 / 1744 : Loss = 20107851.975097656
Batch 800 / 1744 : Loss = 20510042.13098434
Batch 900 / 1744 : Loss = 20381489.28702998
Batch 1000 / 1744 : Loss = 20448826.83607091
Batch 1100 / 1744 : Loss = 20470275.9242062
Batch 1200 / 1744 : Loss = 20394278.18854263
Batch 1300 / 1744 : Loss = 21144531.11735508
Batch 1400 / 1744 : Loss = 21212345.347269956
Batch 1500 / 1744 : Loss = 21150843.23336371
Batch 1600 / 1744 : Loss = 20872878.236714616
Batch 1700 / 1744 : Loss = 20684988.829792146
20610006.9398607
tensor(0.0033)
tensor(0.0485)
tensor(3834.7825)
tensor(21778728.)
Batch 0 / 1744 : Loss = 1304520.875
Batch 100 / 1744 : Loss = 18847345.709090732
Batch 200 / 1744 : Loss = 20060716.454514537
Batch 300 / 1744 : Loss = 19819940.037140198
Batch 400 / 1744 : Loss = 21277384.54472583
Batch 500 / 1744 : Loss = 21803506.13879285
Batch 600 / 1744 : Loss = 21722391.08926295
Batch 700 / 1744 : Loss = 21152319.342875917
Batch 800 / 1744 : Loss = 207

Batch 1100 / 1744 : Loss = 17469310.87729345
Batch 1200 / 1744 : Loss = 17946469.53494499
Batch 1300 / 1744 : Loss = 18128062.4769631
Batch 1400 / 1744 : Loss = 18298614.252171084
Batch 1500 / 1744 : Loss = 18450333.49429006
Batch 1600 / 1744 : Loss = 18242235.03785533
Batch 1700 / 1744 : Loss = 18380764.340183012
18344092.37357264
tensor(0.0025)
tensor(0.0402)
tensor(3233.5007)
tensor(17300810.)
Batch 0 / 1744 : Loss = 16169321.0
Batch 100 / 1744 : Loss = 15692533.498810722
Batch 200 / 1744 : Loss = 19691262.02183905
Batch 300 / 1744 : Loss = 19739775.881935094
Batch 400 / 1744 : Loss = 18380025.03406112
Batch 500 / 1744 : Loss = 17500876.52710314
Batch 600 / 1744 : Loss = 17382511.280949596
Batch 700 / 1744 : Loss = 16931462.61642504
Batch 800 / 1744 : Loss = 16750621.836808832
Batch 900 / 1744 : Loss = 16865172.646675948
Batch 1000 / 1744 : Loss = 16451603.348747566
Batch 1100 / 1744 : Loss = 16663386.553071095
Batch 1200 / 1744 : Loss = 17183424.61385501
Batch 1300 / 1744 : Loss = 

Batch 1600 / 1744 : Loss = 14206963.16542769
Batch 1700 / 1744 : Loss = 14219046.44367693
14109257.079177227
tensor(0.0018)
tensor(0.0349)
tensor(2749.0569)
tensor(11465379.)
Batch 0 / 1744 : Loss = 4385327.0
Batch 100 / 1744 : Loss = 14362428.5616304
Batch 200 / 1744 : Loss = 15528514.38374078
Batch 300 / 1744 : Loss = 15397701.409639986
Batch 400 / 1744 : Loss = 13911785.152631423
Batch 500 / 1744 : Loss = 15219117.081488756
Batch 600 / 1744 : Loss = 14836186.241270043
Batch 700 / 1744 : Loss = 14388200.925722914
Batch 800 / 1744 : Loss = 14097748.071556214
Batch 900 / 1744 : Loss = 14016796.108050788
Batch 1000 / 1744 : Loss = 13698880.265811155
Batch 1100 / 1744 : Loss = 13597741.37375812
Batch 1200 / 1744 : Loss = 13308677.668031259
Batch 1300 / 1744 : Loss = 13592398.23112095
Batch 1400 / 1744 : Loss = 13611208.36159431
Batch 1500 / 1744 : Loss = 13431298.527599398
Batch 1600 / 1744 : Loss = 13399072.122950157
Batch 1700 / 1744 : Loss = 13312740.83492703
13486323.016795272
tensor

Batch 100 / 1744 : Loss = 12361009.533924066
Batch 200 / 1744 : Loss = 12913377.020951455
Batch 300 / 1744 : Loss = 13084525.01262353
Batch 400 / 1744 : Loss = 12998608.41291516
Batch 500 / 1744 : Loss = 12949390.015180698
Batch 600 / 1744 : Loss = 12566636.560316648
Batch 700 / 1744 : Loss = 12921310.034578549
Batch 800 / 1744 : Loss = 12704624.853445902
Batch 900 / 1744 : Loss = 12720976.606377775
Batch 1000 / 1744 : Loss = 13062137.820532922
Batch 1100 / 1744 : Loss = 13096858.72124723
Batch 1200 / 1744 : Loss = 12634777.03451932
Batch 1300 / 1744 : Loss = 12612495.221069945
Batch 1400 / 1744 : Loss = 12591296.005285662
Batch 1500 / 1744 : Loss = 12569002.682206329
Batch 1600 / 1744 : Loss = 12382048.92018481
Batch 1700 / 1744 : Loss = 12300094.997664016
12201979.583270501
tensor(0.0016)
tensor(0.0292)
tensor(2424.5947)
tensor(12103475.)
Batch 0 / 1744 : Loss = 13717533.0
Batch 100 / 1744 : Loss = 13641113.359389503
Batch 200 / 1744 : Loss = 11680738.587931314
Batch 300 / 1744 : Los

Batch 600 / 1744 : Loss = 9738073.225271724
Batch 700 / 1744 : Loss = 9852394.345062038
Batch 800 / 1744 : Loss = 10130717.623841552
Batch 900 / 1744 : Loss = 10012035.733164119
Batch 1000 / 1744 : Loss = 10476914.256617224
Batch 1100 / 1744 : Loss = 10817340.091186468
Batch 1200 / 1744 : Loss = 11131799.387194632
Batch 1300 / 1744 : Loss = 11320924.471140176
Batch 1400 / 1744 : Loss = 11274248.85188349
Batch 1500 / 1744 : Loss = 11470598.706317209
Batch 1600 / 1744 : Loss = 11678706.680743968
Batch 1700 / 1744 : Loss = 11580624.121779382
11556303.490087807
tensor(0.0011)
tensor(0.0243)
tensor(1970.6101)
tensor(7630887.5000)
Batch 0 / 1744 : Loss = 5150701.0
Batch 100 / 1744 : Loss = 9991816.421171585
Batch 200 / 1744 : Loss = 11554830.112234239
Batch 300 / 1744 : Loss = 10604006.870329695
Batch 400 / 1744 : Loss = 9633502.66437544
Batch 500 / 1744 : Loss = 11165801.254567038
Batch 600 / 1744 : Loss = 11593608.258401118
Batch 700 / 1744 : Loss = 12227273.375274528
Batch 800 / 1744 : Lo

Batch 1100 / 1744 : Loss = 11727561.657558348
Batch 1200 / 1744 : Loss = 12015315.904970193
Batch 1300 / 1744 : Loss = 11924553.779496964
Batch 1400 / 1744 : Loss = 11838536.909096478
Batch 1500 / 1744 : Loss = 11533469.027387463
Batch 1600 / 1744 : Loss = 11568206.92297611
Batch 1700 / 1744 : Loss = 11490723.879115334
11463336.889230816
tensor(0.0010)
tensor(0.0243)
tensor(1959.2904)
tensor(7176975.)
Batch 0 / 1744 : Loss = 247692.84375
Batch 100 / 1744 : Loss = 6212691.875558381
Batch 200 / 1744 : Loss = 7683365.95441524
Batch 300 / 1744 : Loss = 10466001.488808667
Batch 400 / 1744 : Loss = 10291602.4253607
Batch 500 / 1744 : Loss = 10223135.31073461
Batch 600 / 1744 : Loss = 9862994.003939662
Batch 700 / 1744 : Loss = 10267486.232043736


In [22]:
model.eval()
output_lst = []
target_lst = []

for batch_idx, data in enumerate(train_dataloader):
    data = data.to(device = device)
    target = data.y.to(device)
    predict = model(data, data.stats)
    output_lst.append(predict.view(-1).item())
    target_lst.append(target.item())

orig_out = torch.FloatTensor(output_lst)
orig_target = torch.FloatTensor(target_lst)

eval_lst = [square_relative_error, mean_relative_error, mean_abs_error, mse]
for evals in eval_lst:
    print(torch.mean(evals(orig_out, orig_target)))

tensor(0.0015)
tensor(0.0300)
tensor(2487.7742)
tensor(11297301.)


In [ ]:
tensor(0.0013)
tensor(0.0290)
tensor(2314.1680)
tensor(9004545.)

In [50]:
orig_out = torch.FloatTensor(scaler.inverse_transform(output_lst))
orig_target = torch.FloatTensor(scaler.inverse_transform(target_lst))

NameError: name 'scaler' is not defined

In [51]:
from sklearn.metrics import mean_absolute_percentage_error

In [36]:
# Sklearn's MAPE
mean_absolute_percentage_error(target_lst, output_lst)

2.656150472792609

In [37]:
orig_out = torch.FloatTensor(output_lst)
orig_target = torch.FloatTensor(target_lst)

In [38]:
# My own version's MAPE
torch.mean(mean_relative_error(orig_out, orig_target))

tensor(2.6562)

In [18]:
scaler = StandardScaler()
Y_orig = np.load("Y_orig.npy")
Y = scaler.fit_transform(Y_orig.reshape(-1, 1)).flatten()

In [30]:
orig_out = torch.FloatTensor(scaler.inverse_transform(output_lst))
orig_target = torch.FloatTensor(scaler.inverse_transform(target_lst))

In [31]:
eval_lst = [square_relative_error, mean_relative_error, mean_abs_error, mse]
for evals in eval_lst:
    print(torch.mean(evals(orig_out, orig_target)))

tensor(0.0058)
tensor(0.0639)
tensor(4922.2793)
tensor(34558900.)


In [57]:
torch.save(model,"train_signnet_util.pt")